In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from gmap_keys import gmap_key

Store Part 1 results into DataFrame

In [2]:
csv_path = os.path.join("city_data.csv")
city_data_df = pd.read_csv(csv_path)
city_data_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Clouds,Winds,Country,Date
0,0,khatanga,71.97,102.50,28.54,98,100,10.29,RU,1601519973
1,1,ilulissat,69.22,-51.10,19.40,57,0,6.93,GL,1601519974
2,2,punta arenas,-53.15,-70.92,32.00,90,20,18.34,CL,1601519974
3,3,east london,-33.02,27.91,62.17,79,80,18.70,ZA,1601519975
4,4,quatre cocos,-20.21,57.76,70.00,93,40,5.82,MU,1601519975


Humidity Heatmap

In [3]:
# Setting up for heatmap
locations = city_data_df[["Latitude", "Longitude"]]

rating = city_data_df["Humidity"].astype(float)

In [4]:
# Showing heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=rating, dissipating=False, max_intensity=10, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

Creating new DataFrame fitting weather criteria

In [6]:
new_cities_df = pd.DataFrame(city_data_df, columns = ["City", "Temperature", "Winds", "Clouds"])
temperature = (new_cities_df["Temperature"] <= 80) & (new_cities_df["Temperature"] > 70)
winds = new_cities_df["Winds"] > 10
clouds = new_cities_df["Clouds"] == 0

new_cities_df[temperature & winds & clouds]

,City,Temperature,Winds,Clouds
132,sao domingos,79.99,12.03,0
147,carnarvon,71.60,19.46,0
188,sao joao da barra,73.98,18.79,0
349,whyalla,73.40,11.41,0


Hotel Map

In [7]:
# Creating new dataframe
hotel_df = pd.DataFrame(city_data_df, columns=["City", "Country", "Latitude", "Longitude"])

hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
0,khatanga,RU,71.97,102.50,
1,ilulissat,GL,69.22,-51.10,
2,punta arenas,CL,-53.15,-70.92,
3,east london,ZA,-33.02,27.91,
4,quatre cocos,MU,-20.21,57.76,


In [8]:
# Setting up url for hotels
target_hotel_coordinates = f"{hotel_df['Latitude'][0]},{hotel_df['Longitude'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

params = {"location": target_hotel_coordinates,
         "keyword": target_search,
         "radius": target_radius,
         "type": target_type,
         "key": gmap_key}


base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

response = requests.get(base_url, params=params)

pprint(response.json(), depth=1)

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}


In [9]:
# Pulling url
hotel_name = []
params = {"radius": 5000,
         "types": "hotel",
         "keyword": "hotel",
         "key": gmap_key}

for index, row in hotel_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    
    params["location"] = f"{latitude},{longitude}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    
    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)
        
hotel_name    

[nan,
 'Hotel Arctic',
 'Hotel Cabo de Hornos',
 'Premier Hotels & Resorts',
 "Shangri-La's Le Touessrok Resort & Spa",
 "Land's End Boutique Hotel",
 "Ellie's Villa",
 'HOTEL GB',
 'Hôtel Hiva Oa Hanakee Lodge',
 'Los Cauquenes Resort & Spa',
 'Café de Upernavik',
 'Pension Bianca & Benoit',
 'Welk Resorts Cabo San Lucas - Sirena del Mar',
 nan,
 nan,
 'Pebbles Inn',
 'One Marine Drive Boutique Hotel',
 'Top of the World Hotel',
 'Abbey Beach Resort',
 'The Peninsula Hong Kong',
 'Pacific Resort Rarotonga',
 nan,
 nan,
 nan,
 'Atlanticview Cape Town Boutique Hotel',
 'LÜDERITZ NEST HOTEL',
 'Hilton Garden Inn Kauai Wailua Bay',
 nan,
 'Royal St Andrews Hotel',
 'Islazul Miraflores',
 nan,
 'Fosshotel Húsavík',
 'Paradise First Hotel',
 'Lanting Hotel',
 'Esperance Chalet Village',
 'Luminoso Lodge',
 'Les Embruns Du Baril',
 'Ghaseali Island Resort',
 'Central Palace Hotel - Castro (PR)',
 'Atlantis Hotel',
 'Hotel Narsaq',
 'Heritage Hotel',
 'Maui Beach Hotel',
 '石梅湾艾美度假酒店',
 nan,
 

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heatmap
fig.add_layer(gmaps.heatmap_layers(locations))
fig = gmaps.figure(layout={
    'width': '400px',
    'height': '600px',
    'padding': '3px',
    'border': '1px solid black'
})

AttributeError: module 'gmaps' has no attribute 'heatmap_layers'

In [13]:
# Display figure
fig = gmaps.figure(map_type='HYBRID')
fig

Figure(layout=FigureLayout(height='420px'))